In [1]:
from mand.core import Entity, node, getNode
from mand.core import ObjectDb, Timestamp, Context, _tr
from mand.core import ProfileMonitor, SummaryMonitor
from mand.core import find, addFootnote
from mand.demos.trading import makeWorld, bookSomeTrades, PnLExplainReport
from mand.lib.dbsetup import setUpDb

db = ObjectDb()
setUpDb(db)

In [2]:
with db:
    pWorld = makeWorld()
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
p4 = pAll.children()[3]

b2 = p2.children()[0]
b4 = p4.children()[0]

makeWorld, TopOfTheHouse is: <Entity:/Global/TradingPortfolio/TopOfTheHouse>
    # books: 100
    # children: 10


In [3]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld)

p1.setChildren(p1.children() + [b2])
ts7 = Timestamp()

# Sanity check of a computation we might want to optimize [Test]

The PnL report is a good test, but at 152K getValues, it's a bit too big to immediately use:

In [4]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

r = PnLExplainReport(valuable=p, ts1=eod, ts2=ts7)

with ProfileMonitor():
    with SummaryMonitor():
        r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# Compute activity summary (17.36 seconds of wall clock time)

|key|value|
|-|-|
|Context|11
|Db.Get|2176
|GetValue|152158
|GetValue/Calc|2891


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|17,352,117|19|17,352,117|GetValue
|PnLExplainReport:data|1|17,352,097|466|17,352,097|GetValue/Calc
|TradingContainer:NPV|11|17,290,362|140|1,571,851|GetValue
|TradingContainer:NPV|11|17,290,222|90|1,571,838|GetValue/Calc
|Portfolio:items|121|17,148,273|1,238|141,721|GetValue
|Portfolio:items|121|17,148,172|836|141,720|GetValue/Calc
|Workbook:items|1,104|16,291,932|11,296|14,757|GetValue
|Workbook:items|1,100|16,280,636|8,502|14,800|GetValue/Calc
|PnLExplainReport:cutoffs|1|9,486,394|19|9,486,394|GetValue
|PnLExplainReport:cutoffs|1|9,486,375|177|9,486,375|GetValue/Calc
|Root:Clocks|548|9,486,102|1,778,244|17,310|Context
|TradeOpenEvent|1,014|3,935,155|3,935,155|3,880|Db.Get
|TradeOpenEvent:ticket|22,286|3,246,092|117,636|145|GetValue
|TradingTicket|1,014|3,128,456|3,128,456|3,085|Db.Get
|Root:Activity Portfolio|263|1,025,349|949,454|3,898|Context
|Root:Amend Trading|255|987,178|911,222|3,871|Context
|Root:Amend MarketData|255|950,186|724,833|3,726|Context
|Root:Start breaks|255|919,655|842,674|3,606|Context
|Portfolio:children|242|817,417|1,604|3,377|GetValue
|Portfolio:children|121|815,813|877|6,742|GetValue/Calc
|Root:End|278|804,567|730,975|2,894|Context
|Root:Activity Trading|263|798,818|724,747|3,037|Context
|Root:Start|270|798,677|725,737|2,958|Context
|Root:Activity MarketData|255|795,306|719,569|3,118|Context
|Root:Amend Portfolio|255|779,738|706,446|3,057|Context
|PortfolioUpdateEvent:children|121|346,174|2,583|2,860|GetValue
|TradingBook|102|318,845|318,845|3,125|Db.Get
|_WorkItemEvent:book2|22,286|264,310|257,578|11|GetValue
|Equity:NPV|15|139,909|164|9,327|GetValue
|Equity:NPV|15|139,744|110|9,316|GetValue/Calc
|MarketInterface:spot|15|132,454|136|8,830|GetValue
|MarketInterface:spot|15|132,317|107|8,821|GetValue/Calc
|ExternalRefData:state|15|124,582|132|8,305|GetValue
|ExternalRefData:state|15|124,449|101|8,296|GetValue/Calc
|RefData:state|15|124,003|132|8,266|GetValue
|RefData:state|15|123,871|105|8,258|GetValue/Calc
|_WorkItemEvent:book1|22,286|109,482|109,482|4|GetValue
|TradeOpenEvent:quantity|22,286|99,231|99,231|4|GetValue
|Clock:cutoffs|2,480|91,268|9,130|36|GetValue
|Clock:cutoffs|20|82,207|150|4,110|GetValue/Calc
|Clock:parent|20|81,317|190|4,065|GetValue
|Clock:parent|20|81,126|154|4,056|GetValue/Calc
|_WorkItemEvent:item|11,143|79,890|72,577|7|GetValue
|TradeOpenEvent:action|11,143|73,160|73,160|6|GetValue
|TradeOpenEvent:premium|11,143|70,994|66,817|6|GetValue
|TradeOpenEvent:unitPrice|11,143|59,837|59,837|5|GetValue
|Event:amends|11,340|57,520|57,520|5|GetValue
|TradingBook:clock|2,200|51,151|18,809|23|GetValue
|PortfolioUpdateEvent|12|47,556|47,556|3,963|Db.Get
|RefDataUpdateEvent|9|32,874|32,874|3,652|Db.Get
|TradingBook:clock|1,100|32,341|10,658|29|GetValue/Calc
|TradingPortfolio|10|31,477|31,477|3,147|Db.Get
|Clock|5|17,490|17,490|3,498|Db.Get
|Portfolio:clock|242|8,693|2,090|35|GetValue
|Portfolio:books|231|7,509|1,663|32|GetValue
|Equity|2|7,313|7,313|3,656|Db.Get
|ClockEvent|2|7,246|7,246|3,623|Db.Get
|MarketInterface:source|15|6,943|124|462|GetValue
|MarketInterface:source|15|6,819|103|454|GetValue/Calc
|Equity:refdata|15|6,653|145|443|GetValue
|Portfolio:clock|121|6,602|1,154|54|GetValue/Calc
|Equity:refdata|15|6,507|103|433|GetValue/Calc
|Portfolio:books|121|6,219|857|51|GetValue/Calc
|ClockEvent:parent|8|6,123|88|765|GetValue
|MarketInterface|2|5,878|5,878|2,939|Db.Get
|MarketDataSource|2|5,791|5,791|2,895|Db.Get
|RootClock|1|5,733|5,733|5,733|Db.Get
|ForwardCashflow|1|4,177|4,177|4,177|Db.Get
|MarketDataSource:clock|30|3,799|240|126|GetValue
|MarketDataSource:clock|15|3,559|141|237|GetValue/Calc
|Entity:clock|40|864|371|21|GetValue
|Entity:clock|20|493|195|24|GetValue/Calc
|MarketInterface:sourceName|15|332|142|22|GetValue
|ForwardCashflow:NPV|11|328|134|29|GetValue
|RefDataUpdateEvent:data|58|273|273|4|GetValue
|RootClock:cutoffs|53|266|195|5|GetValue
|ForwardCashflow:NPV|11|193|79|17|GetValue/Calc
|MarketInterface:sourceName|15|189|113|12|GetValue/Calc
|Equity:assetName|15|104|104|6|GetValue
|RootClock:cutoffs|1|71|36|71|GetValue/Calc
|RootClock:cosmicAll|1|20|10|20|GetValue
|PnLExplainReport:valuable|2|14|14|7|GetValue
|CosmicAll:dbState|1|14|9|14|GetValue
|PnLExplainReport:ts1|2|11|11|5|GetValue
|PnLExplainReport:ts2|2|11|11|5|GetValue
|RootClock:cosmicAll|1|9|9|9|GetValue/Calc
|CosmicAll:dbState|1|5|5|5|GetValue/Calc

# Caching/Reusing results

Until now, we have just checked to see if the current context contains a value for a node we are asking for, and if so, reuse that.

A better approach to caching bound *fn* on *object* in *context0* is:
    
1. Has fn been tweaked in context0? If so, return that
2. Is fn sufficiently trivial that we can avoid managing it as a node?
  * If trivial, just treat it as a pure python function and call it
  * Meta-data (inputs, outputs, etc) will be given to its caller and callees as appropriate
3. Ask object for context1
  * *context1* is a simplified version of *context0*
  * The default case is just to return *context1*
  * IRL, this could actually be a list of contexts or a pattern to match contexts against
4. Is fn cached in *context1*? If so, use that
5. Compute *fn* in *context1*
6. Construct *context2* from the inputs of the computed value
7. If *context2* is a subset of *context1*, cache *fn* in *context1*
8. Something odd happened
  * Footnote the problem as part of computation notes on the node's metadata
  * Maybe cache the node anyway in *context2*
    * Perhaps *object* will return *context2* as a siplification for future computations?
9. Return the value

Notes:
* Parallel compute of nodes not considered yet
* Context simplification (step 3) and input simplification (step 9) are probably intimately related
* The split between calculation and caching is nice:
  * BAs can write business logic
  * Computer scientists can add caching logic where needed
* We can use a profiler type object to gather runtime compute cost infomation
  * The resulting trace can be used as input to drive step 2 and step 3
    

In [5]:
dba = db.copy()
valuable = dba.get(b4.meta.path())

clock = valuable.getObj(_tr.RootClock, 'Main')
with SummaryMonitor():
    with Context({clock.cutoffs: ts7}):
        print valuable.NPV()
        node = getNode(valuable.NPV)
node.printInputGraph()

40.85


# Compute activity summary (0.14 seconds of wall clock time)

|key|value|
|-|-|
|Context|1
|Db.Get|35
|GetValue|185
|GetValue/Calc|24

<Node: /Global/TradingBook/Rates0.(TradingContainer:NPV)() in Root:4485922712 @4785282512 T=False>, nIn=3
   <Node: /Global/Equity/2377c7fd-196f-4313-9b60-9c449acaf560.(Equity:NPV)() in Root:4485922712 @4596269712 T=False>, nIn=2
      <Node: /Global/Equity/2377c7fd-196f-4313-9b60-9c449acaf560.(Equity:refdata)() in Root:4485922712 @4596269840 T=False>, nIn=1
         <Node: /Global/Equity/2377c7fd-196f-4313-9b60-9c449acaf560.(Equity:assetName)() in Root:4485922712 @4596269968 T=False>, nIn=0
      <Node: /Global/MarketInterface/GOOG.(MarketInterface:spot)() in Root:4485922712 @4596270736 T=True>, nIn=2
         <Node: /Global/MarketDataSource/source1.GOOG.(ExternalRefData:state)() in Root:4485922712 @4596272016 T=False>, nIn=1
            <Node: /Global/MarketDataSource/source1.GOOG.(RefData:state)() in Root:4485922712 @4596272400 T=False>, nIn=10
               <Node: /Global/Clock/MarketData.(Clock:cutoffs)() in Root:4485922712 @4573765072 T=True>, nIn=2
                  <Node: /Glo

# Graph Simplification

* Very simple input management simplifier: ignore inputs if they are leaves and not tweakable
* Simple context simplification: just switch to a simple context for certain known methods

In [6]:
from mand.graph import DependencyManager, setDependencyManager
from mand.core import Event

class DM1(DependencyManager):
    
    def __init__(self):
        self.contexts = {}
        self.simpleClockMethods = ('Workbook:items', 'RefData:state', 'Portfolio:children', 'Portfolio:items')
        super(DM1, self).__init__()
        
    def addDep(self, input, output):
        if not input.key.tweakable:
            if not input.inputs:
                return
        output.inputs.add(input)
        input.outputs.add(output)


    def calculated(self, node):
        if not node.isSimplified:
            return
        for input in node.inputs:
            if input not in node.ctx.tweaks:
                addFootnote(text='context simplification failure', info='%s used %s' % (str(node.key), str(input.key)))
                  
    def getCtx(self, clock):
        ts = clock.cutoffs()
        cKey = (clock, ts) # really want value-based comparison of ts, not object equality
        if cKey not in self.contexts:
            self.contexts[cKey] = Context({clock.cutoffs: ts})
        return self.contexts[cKey]
            
    def getNode(self, ctx, key):
        n = ctx._get(key)
        if n:
            return n
        if key.fullName() in self.simpleClockMethods:
            obj = key.object()
            ctx1 = self.getCtx(obj.clock())
            ret = ctx1.get(key)
            ret.isSimplified = True
        else:
            ret = super(DM1, self).getNode(ctx, key)
            ret.isSimplified = False
        return ret
    
setDependencyManager(DM1())

db4 = db.copy()
valuable = db4.get(b4.meta.path())

clock = valuable.getObj(_tr.RootClock, 'Main')
with Context({clock.cutoffs: ts7}):
    print valuable.NPV()
    node = getNode(valuable.NPV)
    node.printInputGraph()

40.85
<Node: /Global/TradingBook/Rates0.(TradingContainer:NPV)() in Root:4485922712 @4654306256 T=False>, nIn=3
   <Node: /Global/Clock/Trading.(Clock:cutoffs)() in Root:4485922712 @4674287888 T=True>, nIn=2
      <Node: /Global/Clock/Trading.(Clock:parent)() in Root:4485922712 @4674287952 T=False>, nIn=1
         <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4485922712 @4654306064 T=True>, nIn=0
      <Node: /Global/Clock/Workflow.(Clock:cutoffs)() in Root:4485922712 @4654308304 T=True>, nIn=2
         <Node: /Global/Clock/Workflow.(Clock:parent)() in Root:4485922712 @4654308176 T=False>, nIn=1
            <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4485922712 @4654306064 T=True>, nIn=0
         <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4485922712 @4654306064 T=True>, nIn=0
   <Node: /Global/Equity/2377c7fd-196f-4313-9b60-9c449acaf560.(Equity:NPV)() in Root:4485922712 @4585100176 T=False>, nIn=1
      <Node: /Global/MarketInterface/GOOG.

In [7]:
db4 = db.copy()
valuable = db4.get(pAll.meta.path())

r = PnLExplainReport(valuable=valuable, ts1=eod, ts2=ts7)
with ProfileMonitor():
    with SummaryMonitor():
        r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|0.00
|activity: MarketData|5240.00
|activity: Portfolio|1016.00
|activity: Trading|0.00
|Ending balance breaks|0.00

1. context simplification failure
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta10>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta11>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta12>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta13>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta14>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta15>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta16>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta17>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta18>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta19>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:children) ()>
1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# Compute activity summary (9.20 seconds of wall clock time)

|key|value|
|-|-|
|Context|19
|Db.Get|2176
|GetValue|28897
|GetValue/Calc|1031


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|9,190,831|14|9,190,831|GetValue
|PnLExplainReport:data|1|9,190,817|414|9,190,817|GetValue/Calc
|TradingContainer:NPV|11|9,183,198|125|834,836|GetValue
|TradingContainer:NPV|11|9,183,073|85|834,824|GetValue/Calc
|PnLExplainReport:cutoffs|1|9,108,361|15|9,108,361|GetValue
|PnLExplainReport:cutoffs|1|9,108,345|150|9,108,345|GetValue/Calc
|Root:Clocks|47|9,108,102|23,313|193,789|Context
|Portfolio:items|41|9,062,564|5,743|221,038|GetValue
|Portfolio:items|33|9,062,153|231|274,610|GetValue/Calc
|Root:Clocks:4485922712|306|9,057,461|110,995|29,599|Context
|Workbook:items|301|8,528,995|2,330|28,335|GetValue
|Workbook:items|200|8,526,665|1,440|42,633|GetValue/Calc
|Root:Clocks:4485922712:4485922712|400|8,525,224|1,381,506|21,313|Context
|TradeOpenEvent|1,014|3,641,757|3,641,757|3,591|Db.Get
|TradeOpenEvent:ticket|4,052|3,358,099|28,190|828|GetValue
|TradingTicket|1,014|3,329,908|3,329,908|3,283|Db.Get
|Portfolio:children|66|475,061|512|7,197|GetValue
|Portfolio:children|33|474,549|252|14,380|GetValue/Calc
|PortfolioUpdateEvent:children|33|321,646|1,312|9,746|GetValue
|TradingBook|102|296,778|296,778|2,909|Db.Get
|Equity:NPV|15|100,033|152|6,668|GetValue
|Equity:NPV|15|99,881|98|6,658|GetValue/Calc
|MarketInterface:spot|15|91,155|142|6,077|GetValue
|MarketInterface:spot|15|91,013|100|6,067|GetValue/Calc
|ExternalRefData:state|15|83,510|132|5,567|GetValue
|ExternalRefData:state|15|83,378|102|5,558|GetValue/Calc
|Clock:cutoffs|870|68,354|3,159|78|GetValue
|Clock:cutoffs|16|65,251|112|4,078|GetValue/Calc
|Clock:parent|16|64,501|145|4,031|GetValue
|Clock:parent|16|64,355|120|4,022|GetValue/Calc
|Root:Amend Portfolio|11|48,394|459|4,399|Context
|Root:Amend Portfolio:4485922712|144|47,616|42,216|330|Context
|RefData:state|15|46,968|81|3,131|GetValue
|RefData:state|4|46,886|30|11,721|GetValue/Calc
|PortfolioUpdateEvent|12|44,652|44,652|3,721|Db.Get
|RefDataUpdateEvent|9|32,030|32,030|3,558|Db.Get
|TradingPortfolio|10|29,593|29,593|2,959|Db.Get
|_WorkItemEvent:book2|4,052|25,902|19,864|6|GetValue
|_WorkItemEvent:item|2,026|21,677|15,870|10|GetValue
|_WorkItemEvent:book1|4,052|20,714|20,714|5|GetValue
|TradingBook:clock|701|19,349|6,311|27|GetValue
|TradeOpenEvent:quantity|4,052|18,383|18,383|4|GetValue
|TradeOpenEvent:premium|2,026|15,532|12,573|7|GetValue
|Clock|5|15,457|15,457|3,091|Db.Get
|TradeOpenEvent:action|2,026|13,913|13,913|6|GetValue
|TradingBook:clock|500|13,037|4,142|26|GetValue/Calc
|TradeOpenEvent:unitPrice|2,026|10,929|10,929|5|GetValue
|Root:Start|20|10,719|10,309|535|Context
|Event:amends|2,083|10,490|10,490|5|GetValue
|Root:End|27|10,399|9,906|385|Context
|Equity:refdata|15|8,086|145|539|GetValue
|Equity:refdata|15|7,941|106|529|GetValue/Calc
|MarketInterface|2|7,224|7,224|3,612|Db.Get
|MarketInterface:source|15|6,834|131|455|GetValue
|MarketInterface:source|15|6,702|95|446|GetValue/Calc
|ClockEvent|2|6,471|6,471|3,235|Db.Get
|ClockEvent:parent|6|5,971|61|995|GetValue
|MarketDataSource|2|5,878|5,878|2,939|Db.Get
|Equity|2|5,806|5,806|2,903|Db.Get
|Portfolio:clock|140|5,472|916|39|GetValue
|Portfolio:clock|44|4,555|328|103|GetValue/Calc
|Root:Amend MarketData|11|4,529|428|411|Context
|RootClock|1|4,402|4,402|4,402|Db.Get
|Root:Amend MarketData:4485922712|2|3,906|3,833|1,953|Context
|MarketDataSource:clock|23|3,468|199|150|GetValue
|MarketDataSource:clock|19|3,268|137|172|GetValue/Calc
|ForwardCashflow|1|2,959|2,959|2,959|Db.Get
|Portfolio:books|63|1,940|449|30|GetValue
|Portfolio:books|33|1,595|219|48|GetValue/Calc
|Root:Activity Portfolio|18|996|702|55|Context
|Root:Activity Trading|18|882|596|49|Context
|Entity:clock|32|628|269|19|GetValue
|Root:Start breaks|11|589|411|53|Context
|Root:Amend Trading|11|557|385|50|Context
|Root:Activity MarketData|11|547|376|49|Context
|Entity:clock|16|359|152|22|GetValue/Calc
|MarketInterface:sourceName|15|303|130|20|GetValue
|ForwardCashflow:NPV|11|297|118|27|GetValue
|RootClock:cutoffs|43|230|159|5|GetValue
|ForwardCashflow:NPV|11|178|74|16|GetValue/Calc
|MarketInterface:sourceName|15|172|99|11|GetValue/Calc
|Equity:assetName|15|88|88|5|GetValue
|RefDataUpdateEvent:data|15|77|77|5|GetValue
|RootClock:cutoffs|1|71|37|71|GetValue/Calc
|RootClock:cosmicAll|1|19|11|19|GetValue
|CosmicAll:dbState|1|14|10|14|GetValue
|PnLExplainReport:ts2|2|12|12|6|GetValue
|PnLExplainReport:ts1|2|12|12|6|GetValue
|PnLExplainReport:valuable|2|10|10|5|GetValue
|RootClock:cosmicAll|1|7|7|7|GetValue/Calc
|CosmicAll:dbState|1|4|4|4|GetValue/Calc